In [ ]:
import os
import sys
import wandb
import torch
import GPUtil
from models.eegnet.EEGNet import EEGNet
from torchinfo import summary
from torch.utils.data import DataLoader

sys.path.append('../../')
from utils.coco_data_handler import COCODataHandler
from utils.hdf5_data_split_generator import HDF5DataSplitGenerator

s:\PolySecLabProjects\eeg-image-decode\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # Force CUDA to use the GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use first GPU
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" # Enable memory optimization settings for PyTorch

In [ ]:
# Check if CUDA is available
try:
    gpus = GPUtil.getGPUs()
    if gpus:
        print(f"GPUtil detected {len(gpus)} GPUs:")
        for i, gpu in enumerate(gpus):
            print(f"  GPU {i}: {gpu.name} (Memory: {gpu.memoryTotal}MB)")
        
        # Set default GPU
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in range(len(gpus))])
        print(f"Set CUDA_VISIBLE_DEVICES={os.environ['CUDA_VISIBLE_DEVICES']}")
    else:
        print("GPUtil found no available GPUs")
except Exception as e:
    print(f"Error checking GPUs with GPUtil: {e}")

In [ ]:
# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Print available GPU memory
if torch.cuda.is_available():
    print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Available GPU memory: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

In [5]:
coco_data = COCODataHandler.from_file('../../../data/all-joined-1/coco-train17-captions-and-categories.json')
coco_data.category_index

{'accessory': 0,
 'animal': 1,
 'appliance': 2,
 'electronic': 3,
 'food': 4,
 'furniture': 5,
 'indoor': 6,
 'kitchen': 7,
 'outdoor': 8,
 'person': 9,
 'sports': 10,
 'vehicle': 11}

In [ ]:
all_channels = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

# Model parameters
hr_channel_names = all_channels # High-resolution setup (all channels)
lr_channel_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4'] # Low-resolution setup (fewer channels)
num_classes = len(coco_data.category_index.keys())

# Training parameters
epochs = 30

# Optimizer parameters
lr = 5e-5
weight_decay = 0.5
beta_1 = 0.9
beta_2 = 0.95

# Dataset parameters
batch_size = 30
dataset_split = "70/25/5"
eeg_epoch_mode = "around_evoked_event"
duration_before_onset = 0.05
duration_after_onset = 0.6
random_state = 97

In [ ]:
train_dataset = HDF5DataSplitGenerator(
    dataset_type="train",
    dataset_split=dataset_split,
    eeg_epoch_mode=eeg_epoch_mode,
    duration_before_onset=duration_before_onset,
    duration_after_onset=duration_after_onset
)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

len(train_loader)

In [ ]:
input_channels, input_window_samples = train_loader.dataset[0]['lo_res'].shape

In [ ]:
model = EEGNet(input_channels, input_window_samples, num_classes)
summary(model)

In [ ]:
history = model.fit(train_loader, coco_data, epochs)